In [1]:
import numpy as np

np.show_config()

Build Dependencies:
  blas:
    detection method: pkgconfig
    found: true
    include directory: /opt/arm64-builds/include
    lib directory: /opt/arm64-builds/lib
    name: openblas64
    openblas configuration: USE_64BITINT=1 DYNAMIC_ARCH=1 DYNAMIC_OLDER= NO_CBLAS=
      NO_LAPACK= NO_LAPACKE= NO_AFFINITY=1 USE_OPENMP= SANDYBRIDGE MAX_THREADS=3
    pc file directory: /usr/local/lib/pkgconfig
    version: 0.3.23.dev
  lapack:
    detection method: internal
    found: true
    include directory: unknown
    lib directory: unknown
    name: dep4350285776
    openblas configuration: unknown
    pc file directory: unknown
    version: 1.26.4
Compilers:
  c:
    args: -fno-strict-aliasing, -DBLAS_SYMBOL_SUFFIX=64_, -DHAVE_BLAS_ILP64
    commands: cc
    linker: ld64
    linker args: -fno-strict-aliasing, -DBLAS_SYMBOL_SUFFIX=64_, -DHAVE_BLAS_ILP64
    name: clang
    version: 14.0.0
  c++:
    args: -DBLAS_SYMBOL_SUFFIX=64_, -DHAVE_BLAS_ILP64
    commands: c++
    linker: ld64
    linker

# side model

In [2]:
import pandas as pd

df_features = pd.read_parquet("data/features_45m.parquet")
label_side = np.load("data/label_dollar_bar_45m.npy")
print(df_features.shape)
print(label_side.shape)

df_features.isna().sum().sort_values(ascending=False)

(59137, 11841)
(59137, 7)


4h_williams_r_ddt_lag39         538
4h_adaptive_stochastic_lag28    538
4h_adaptive_stochastic_lag21    538
4h_adaptive_stochastic_lag22    538
4h_adaptive_stochastic_lag23    538
                               ... 
45m_comb_spectrum_pwr_38          0
45m_comb_spectrum_pwr_37          0
45m_comb_spectrum_pwr_36          0
45m_comb_spectrum_pwr_35          0
45m_comb_spectrum_pwr_28          0
Length: 11841, dtype: int64

In [3]:
df_features = df_features[538:]
label_side = label_side[538:]

print(df_features.shape)
print(label_side.shape)

label_side

(58599, 11841)
(58599, 7)


array([[ 1.58741034e+12,  6.80359000e+03,  6.85246000e+03, ...,
         6.73595000e+03,  5.71097240e+04,  1.00000000e+00],
       [ 1.58741640e+12,  6.85222000e+03,  6.86764000e+03, ...,
         6.80948000e+03,  5.52900840e+04,  1.00000000e+00],
       [ 1.58743416e+12,  6.86765000e+03,  6.83300000e+03, ...,
         6.82600000e+03,  5.53223140e+04,  1.00000000e+00],
       ...,
       [ 1.73826966e+12,  1.05086300e+05,  1.04836200e+05, ...,
         1.04768000e+05,  3.81996600e+03, -1.00000000e+00],
       [ 1.73827080e+12,  1.04836300e+05,  1.05080900e+05, ...,
         1.04737000e+05,  3.65874900e+03, -1.00000000e+00],
       [ 1.73827656e+12,  1.05080800e+05,  1.04676900e+05, ...,
         1.04628300e+05,  3.65185700e+03, -1.00000000e+00]])

In [4]:
from custom_indicators.config import SIDE_ALL

side_features = df_features[SIDE_ALL]
side_label = (label_side[:, 6].astype(int) == 1).astype(int)
print(np.unique(side_label, return_counts=True))

train_test_split_point = int(side_features.shape[0] * 0.8)
side_features_train = side_features[:train_test_split_point]
side_features_test = side_features[train_test_split_point:]
side_label_train = side_label[:train_test_split_point]
side_label_test = side_label[train_test_split_point:]

print(np.unique(side_label_test, return_counts=True))

(array([0, 1]), array([28492, 30107]))
(array([0, 1]), array([5634, 6086]))


In [5]:
import lightgbm as lgb
import optuna
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score


def objective(trial):
    params = {
        "objective": "binary",
        "metric": "auc",
        "is_unbalance": True,
        "num_threads": -1,
        "verbose": -1,
        "extra_trees": trial.suggest_categorical("extra_trees", [True, False]),
        # "early_stopping_round": 100,
        "boosting": trial.suggest_categorical("boosting", ["gbdt", "dart"]),
        "num_leaves": trial.suggest_int("num_leaves", 31, 300),
        "max_depth": trial.suggest_int("max_depth", 30, 100),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 1e-8, 1),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 20, 300),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-4, 100),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-4, 100),
    }
    dtrain = lgb.Dataset(side_features_train, side_label_train)
    dtest = lgb.Dataset(side_features_test, side_label_test)
    model = lgb.train(
        params,
        dtrain,
        # valid_sets=[dtest],
        num_boost_round=trial.suggest_int("num_boost_round", 100, 800),
    )
    pred_proba = model.predict(side_features_test)
    pred_label = (pred_proba > 0.5).astype(int)
    precision = precision_score(side_label_test, pred_label)
    recall = recall_score(side_label_test, pred_label)
    f1 = f1_score(side_label_test, pred_label)
    auc = roc_auc_score(side_label_test, pred_proba)
    print(f"precision: {precision}, recall: {recall}, f1: {f1}")
    return auc


study = optuna.create_study(
    direction="maximize",
    pruner=optuna.pruners.HyperbandPruner(),
    sampler=optuna.samplers.TPESampler(),
)
study.optimize(objective, n_trials=50, n_jobs=1)

/opt/homebrew/Caskroom/miniforge/base/envs/jesse/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-03-06 14:45:53,723] A new study created in memory with name: no-name-f6503f6a-e169-4e83-8aa9-4d9a9469ae02
[I 2025-03-06 14:46:30,537] Trial 0 finished with value: 0.8867318698232679 and parameters: {'extra_trees': False, 'boosting': 'gbdt', 'num_leaves': 185, 'max_depth': 31, 'min_gain_to_split': 0.3954180227350151, 'min_data_in_leaf': 255, 'lambda_l1': 36.79300410582504, 'lambda_l2': 25.195986483701578, 'num_boost_round': 695}. Best is trial 0 with value: 0.8867318698232679.


precision: 0.801798800799467, recall: 0.7909957279000985, f1: 0.7963606286186932


[I 2025-03-06 14:46:49,970] Trial 1 finished with value: 0.8873159719560983 and parameters: {'extra_trees': False, 'boosting': 'gbdt', 'num_leaves': 64, 'max_depth': 69, 'min_gain_to_split': 0.8394890500223642, 'min_data_in_leaf': 158, 'lambda_l1': 81.87846362105464, 'lambda_l2': 66.2845141387739, 'num_boost_round': 366}. Best is trial 1 with value: 0.8873159719560983.


precision: 0.7995361166335322, recall: 0.7929674663161353, f1: 0.7962382445141066


[I 2025-03-06 14:47:28,226] Trial 2 finished with value: 0.8884914964551989 and parameters: {'extra_trees': False, 'boosting': 'dart', 'num_leaves': 105, 'max_depth': 82, 'min_gain_to_split': 0.9971086545069983, 'min_data_in_leaf': 128, 'lambda_l1': 52.25529290265952, 'lambda_l2': 30.666818720994208, 'num_boost_round': 403}. Best is trial 2 with value: 0.8884914964551989.


...

precision: 0.8042319226924358, recall: 0.7931317778508051, f1: 0.7986432825943084


[I 2025-03-06 15:14:46,793] Trial 48 finished with value: 0.8880336173117279 and parameters: {'extra_trees': False, 'boosting': 'dart', 'num_leaves': 111, 'max_depth': 43, 'min_gain_to_split': 0.532547169814198, 'min_data_in_leaf': 285, 'lambda_l1': 17.898945047497726, 'lambda_l2': 11.545261426134573, 'num_boost_round': 739}. Best is trial 31 with value: 0.8928647672323254.


precision: 0.8034188034188035, recall: 0.7877094972067039, f1: 0.7954866008462623


[I 2025-03-06 15:15:08,849] Trial 49 finished with value: 0.886194138890318 and parameters: {'extra_trees': True, 'boosting': 'gbdt', 'num_leaves': 71, 'max_depth': 79, 'min_gain_to_split': 0.7160659567941552, 'min_data_in_leaf': 224, 'lambda_l1': 0.5599676088846604, 'lambda_l2': 0.7487864847936976, 'num_boost_round': 364}. Best is trial 31 with value: 0.8928647672323254.


precision: 0.806200237167542, recall: 0.7819585934932632, f1: 0.793894403202936


In [6]:
study.best_params

{'extra_trees': True,
 'boosting': 'dart',
 'num_leaves': 77,
 'max_depth': 58,
 'min_gain_to_split': 0.19783066518532677,
 'min_data_in_leaf': 199,
 'lambda_l1': 7.778119042472355,
 'lambda_l2': 27.748014574266364,
 'num_boost_round': 480}

In [7]:
params = {
    "is_unbalance": True,
    **study.best_params,
}

dtrain = lgb.Dataset(side_features_train, side_label_train)
model = lgb.train(params, dtrain)

In [8]:
import plotly.express as px

res = model.predict(side_features_train)

fig = px.histogram(res, nbins=100)
fig.show()

In [9]:
import plotly.express as px

res = model.predict(side_features_test)
print(roc_auc_score(side_label_test, res))

fig = px.histogram(res, nbins=100)
fig.show()

0.8898870070931022


In [10]:
model.save_model("custom_indicators/models/model_side.txt")

# meta model

In [12]:
import lightgbm as lgb
import numpy as np
import pandas as pd

from custom_indicators.config import SIDE_ALL

df_features = pd.read_parquet("data/features_45m.parquet")
df_label = pd.read_parquet("data/label_meta_45m.parquet")

model_side = lgb.Booster(model_file="custom_indicators/models/model_side.txt")
model_side_res = model_side.predict(df_features[SIDE_ALL])
df_features["model_side_res"] = model_side_res

In [13]:
df_label["ret"].notna().sum()

14896

In [14]:
df_features = df_features[583:]
df_label = df_label[583:]

print(df_features.shape)

df_features = df_features[df_label["ret"].notna()]
df_label = df_label[df_label["ret"].notna()]

assert df_features.shape[0] == df_label.shape[0]
print(df_label.shape)
df_label.head()

(58554, 11842)
(14736, 9)


,open,high,low,close,volume,ret,trgt,bin,side
index,,,,,,,,,
2020-04-28 21:54:00+00:00,7716.02,7879.99,7712.92,7842.79,48845.750,0.034708,0.010402,1.0,1.0
2020-04-29 02:08:00+00:00,7843.44,7843.44,7710.00,7820.51,48915.848,0.037656,0.010812,1.0,1.0
2020-04-29 04:45:00+00:00,7820.39,7955.00,7819.59,7941.16,48157.749,0.021891,0.010790,1.0,1.0
2020-04-29 06:35:00+00:00,7941.16,7965.99,7909.54,7958.77,49086.039,0.022067,0.009961,1.0,1.0
2020-04-29 09:43:00+00:00,7958.77,8128.00,7940.22,8115.00,47613.448,0.030865,0.010091,1.0,1.0


In [15]:
from custom_indicators.config import META_ALL

meta_features = df_features[META_ALL]
print(meta_features.shape)
meta_label = df_label["bin"].astype(int)

train_test_split_point = int(meta_features.shape[0] * 0.8)
meta_features_train = meta_features.iloc[:train_test_split_point]
meta_features_test = meta_features.iloc[train_test_split_point:]
meta_label_train = meta_label.iloc[:train_test_split_point]
meta_label_test = meta_label.iloc[train_test_split_point:]

meta_label_test.value_counts()

(14736, 4438)


bin
1    1526
0    1422
Name: count, dtype: int64

In [16]:
import lightgbm as lgb
import optuna
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score


def objective(trial):
    params = {
        "objective": "binary",
        # "is_unbalance": True,
        "num_threads": -1,
        "verbose": -1,
        "extra_trees": trial.suggest_categorical("extra_trees", [True, False]),
        "boosting": trial.suggest_categorical("boosting", ["gbdt", "dart"]),
        "num_leaves": trial.suggest_int("num_leaves", 31, 300),
        "max_depth": trial.suggest_int("max_depth", 30, 100),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 1e-8, 1),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 20, 300),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 100),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 100),
    }
    dtrain = lgb.Dataset(meta_features_train, meta_label_train)
    dtest = lgb.Dataset(meta_features_test, meta_label_test)
    model = lgb.train(
        params,
        dtrain,
        num_boost_round=trial.suggest_int("num_boost_round", 100, 1000),
    )
    pred_proba = model.predict(meta_features_test)
    pred_label = (pred_proba > 0.5).astype(int)
    precision = precision_score(meta_label_test, pred_label)
    recall = recall_score(meta_label_test, pred_label)
    f1 = f1_score(meta_label_test, pred_label)
    auc = roc_auc_score(meta_label_test, pred_proba)
    print(f"precision: {precision}, recall: {recall}, f1: {f1}, auc: {auc}")
    return f1


study = optuna.create_study(
    direction="maximize",
    pruner=optuna.pruners.HyperbandPruner(),
    sampler=optuna.samplers.TPESampler(),
)
study.optimize(objective, n_trials=100, n_jobs=1)

[I 2025-03-07 12:46:14,445] A new study created in memory with name: no-name-77ea8ae0-9bd8-438a-ba1f-bcad2c0c7423
[I 2025-03-07 12:46:32,088] Trial 0 finished with value: 0.4182692307692308 and parameters: {'extra_trees': True, 'boosting': 'dart', 'num_leaves': 287, 'max_depth': 42, 'min_gain_to_split': 0.5277602343585173, 'min_data_in_leaf': 106, 'lambda_l1': 92.56704217506919, 'lambda_l2': 28.4784572009264, 'num_boost_round': 900}. Best is trial 0 with value: 0.4182692307692308.


precision: 0.5381443298969072, recall: 0.3420707732634338, f1: 0.4182692307692308, auc: 0.493525492494834


[I 2025-03-07 12:46:36,865] Trial 1 finished with value: 0.3988786543852623 and parameters: {'extra_trees': True, 'boosting': 'gbdt', 'num_leaves': 185, 'max_depth': 49, 'min_gain_to_split': 0.6293765158671873, 'min_data_in_leaf': 167, 'lambda_l1': 14.642195916689658, 'lambda_l2': 27.92793448097844, 'num_boost_round': 161}. Best is trial 0 with value: 0.4182692307692308.


precision: 0.5128733264675592, recall: 0.32634338138925295, f1: 0.3988786543852623, auc: 0.5022184618050372


[I 2025-03-07 12:46:44,121] Trial 2 finished with value: 0.4109697933227345 and parameters: {'extra_trees': True, 'boosting': 'gbdt', 'num_leaves': 268, 'max_depth': 34, 'min_gain_to_split': 0.8162836423499501, 'min_data_in_leaf': 299, 'lambda_l1': 59.59317338076894, 'lambda_l2': 41.91567884385067, 'num_boost_round': 330}. Best is trial 0 with value: 0.4182692307692308.


...

precision: 0.5189075630252101, recall: 0.3237221494102228, f1: 0.3987086359967716, auc: 0.4971688574783453


[I 2025-03-07 13:09:04,265] Trial 98 finished with value: 0.4082960553070354 and parameters: {'extra_trees': True, 'boosting': 'gbdt', 'num_leaves': 227, 'max_depth': 48, 'min_gain_to_split': 0.4406399047694806, 'min_data_in_leaf': 50, 'lambda_l1': 99.7798375814652, 'lambda_l2': 21.752289546133284, 'num_boost_round': 241}. Best is trial 66 with value: 0.4438291139240506.


precision: 0.5380493033226152, recall: 0.3289646133682831, f1: 0.4082960553070354, auc: 0.5029320654828726


[I 2025-03-07 13:09:09,213] Trial 99 finished with value: 0.39476910502656315 and parameters: {'extra_trees': True, 'boosting': 'gbdt', 'num_leaves': 202, 'max_depth': 41, 'min_gain_to_split': 0.5839121615716808, 'min_data_in_leaf': 39, 'lambda_l1': 88.47473446149459, 'lambda_l2': 0.5452426642753405, 'num_boost_round': 211}. Best is trial 66 with value: 0.4438291139240506.


precision: 0.5244299674267101, recall: 0.3165137614678899, f1: 0.39476910502656315, auc: 0.497758726840715


In [17]:
study.best_params

{'extra_trees': True,
 'boosting': 'gbdt',
 'num_leaves': 274,
 'max_depth': 57,
 'min_gain_to_split': 0.6606789380825927,
 'min_data_in_leaf': 101,
 'lambda_l1': 96.27579805878213,
 'lambda_l2': 14.860350102315499,
 'num_boost_round': 1000}

In [18]:
params = {
    "is_unbalance": True,
    **study.best_params,
}

model = lgb.train(
    params,
    lgb.Dataset(meta_features_train, meta_label_train),
)

In [20]:
import plotly.express as px

res = model.predict(meta_features_test)

fig = px.histogram(res, nbins=100)
fig.show()

In [30]:
import plotly.express as px

df_features = pd.read_parquet("data/features_45m.parquet")
model_side = lgb.Booster(model_file="custom_indicators/models/model_side.txt")
model_side_res = model_side.predict(df_features[SIDE_ALL])
df_features["model_side_res"] = model_side_res
df_features = df_features[META_ALL]
df_label = pd.read_parquet("data/label_meta_45m.parquet")

extra_features_test = df_features[int(len(df_features) * 0.8) :]
extra_label = df_label[int(len(df_features) * 0.8) :]

extra_features_test = extra_features_test[df_label["ret"].isna()]
extra_label = extra_label[df_label["ret"].isna()]

assert len(extra_features_test) == len(extra_label)

res = model.predict(extra_features_test)

fig = px.histogram(res, nbins=100)
fig.show()

/var/folders/h9/d_xpszfn6yz9r8zww450m44w0000gn/T/ipykernel_30270/2819802013.py:13: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/var/folders/h9/d_xpszfn6yz9r8zww450m44w0000gn/T/ipykernel_30270/2819802013.py:14: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [21]:
model.save_model("custom_indicators/models/model_meta.txt")